In [1]:
!pip install datasetsforecast hierarchicalforecast

  Using cached datasetsforecast-0.0.7-py3-none-any.whl (22 kB)
  Using cached hierarchicalforecast-0.2.0-py3-none-any.whl (33 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.4/92.4 kB 3.1 MB/s eta 0:00:00
  Using cached mycolorpy-1.5.1.tar.gz (2.5 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for mycolorpy: filename=mycolorpy-1.5.1-py3-none-any.whl size=3873 sha256=8bf6e004aa8e1deb8feeff6ad08737c8f7b4af4ad6b4d7470be8de63920ccfef
  Stored in directory: /Users/kevinkho/Library/Caches/pip/wheels/de/f6/20/d5f4361bb07416d18914807370aac98a7581f644cf18362df0
Successfully built mycolorpy


In [6]:
import numpy as np
import pandas as pd

#obtain hierarchical dataset
from datasetsforecast.hierarchical import HierarchicalData

# Load TourismSmall dataset
Y_df, S, tags = HierarchicalData.load('./data', 'TourismSmall')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

ModuleNotFoundError: No module named 'numpy'

In [ ]:

#split train/test sets
Y_test_df  = Y_df.groupby('unique_id').tail(12)
Y_train_df = Y_df.drop(Y_test_df.index)
Y_test_df  = Y_test_df.set_index('unique_id')
Y_train_df = Y_train_df.set_index('unique_id')

# Compute base auto-ARIMA predictions
fcst = StatsForecast(df=Y_train_df, 
                     models=[AutoARIMA(season_length=12), Naive()], 
                     freq='M', n_jobs=-1)
Y_hat_df = fcst.forecast(h=12)

# Reconcile the base predictions
reconcilers = [
    BottomUp(),
    TopDown(method='forecast_proportions'),
    MiddleOut(middle_level='Country/Purpose/State',
              top_down_method='forecast_proportions')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_train_df, 
                          S=S, tags=tags)

In [ ]:
# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut